In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datenspende.utils import query_ch_df, query_pg_df
import datetime
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import os
import glob
from functools import reduce
import seaborn as sns

In [ ]:
lc_u = np.load('uid_per_shb_fatigue(1).npy')

In [ ]:
path = os.getcwd()+'/user_epoch/'
#path = os.getcwd()+'/extcoh/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

df_lcu = []
demo_info = []
uid = []
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f, index_col = 0)
    #for var in ['sleep', 'hr', 'steps']:
    #    if var not in df.columns:
    #        df[var] = np.nan
    df_lcu.append(df)
    demo_info.append(f.split("/")[-1].split('.')[0])
    uid.append(int(f.split("/")[-1].split('.')[1][1:])) 
      
sex = [re.split('(\d+)',demo_info[i])[0] for i in range(len(demo_info))]
age = [int(re.split('(\d+)',demo_info[i])[1]) for i in range(len(demo_info))] 

In [ ]:
path = os.getcwd()+'/user_epoch/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

uid = []
sex = []
age = []
df_lcu = []
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f, index_col = 0)
    if len(df['phase'].unique()) >= 4 : #and 0 in df['phase'].unique() and 1 in df['phase'].unique()
        if np.all([np.count_nonzero(df[df['phase']==ph]['hr'].notna()) > 100 for ph in df['phase'].unique()]):
                 
            df_lcu.append(df)
            demo_info = f.split("/")[-1].split('.')[0]
            sex.append(re.split('(\d+)',demo_info)[0])
            age.append(int(re.split('(\d+)',demo_info)[1]))
            uid.append(int(f.split("/")[-1].split('.')[1][1:])) 

## functions

In [ ]:
def sliding_window(elements, window_size):
    wi_days = []
    for i in range(1,len(elements)+1):
        if i >= window_size:
            wi_days.append(elements[i-window_size:i])
        elif i < window_size:
            wi_days.append(elements[:i])
    return wi_days

## simple timeseries

In [ ]:
# check_u = np.array([ 601007, 1217724, 1225571,  468824, 1232956,  146789,  198054,
#         297815,  278688, 1220358,  120014, 1208175, 1225369,  865661,
#        1116909,  203461, 1116926, 1245237,  827739,  357322, 1091037,
#           8703,  494935, 1234389,  660061, 1148848,  859881, 1228834,
#          83696, 1217773, 1110229, 1075521,  200221,  789093, 1227939,
#         128235,  713054,  288844])

In [ ]:
NU = [0,1,3,8,10,18,21,23,29,31,33,38,40,43,44,47]
U = [u for u in range(len(df_lcu)) if u not in NU]

In [ ]:
lid = np.where(np.array(uid) == 789093)[0][0]

In [ ]:
#len(U)

In [ ]:
#lid=U[30]

In [ ]:
age[lid]

In [ ]:
sex[lid]

In [ ]:
n = lid
df = df_lcu[n].copy()#dfs[0].copy()
df['start'] = pd.to_datetime(df['start'])

In [ ]:
userid = uid[n]

In [ ]:
userid

In [ ]:
df['dt'] = pd.to_datetime(df['dt'])
df['date'] = pd.to_datetime(df['date'])
mis_row = []
for dt in [d for d in pd.date_range(start = df['date'].min(), end = df['date'].max()).difference(df.index) if d not in df['date'].unique()]:
    mis_row.append([
            np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, dt, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
if len(mis_row) > 0:
    df = df.append(pd.DataFrame(data=mis_row, columns=df.columns))
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')
df['start'] = pd.to_datetime(df['start']) 
dd = df.groupby('date').sum() 

In [ ]:
def get_min(subd):
    #subd['start'] = pd.to_datetime(subd['start'])
    subd = subd[subd['start'].notna()]
    if len(subd) > 0:
        night_inact = subd[(subd['start'].dt.time < pd.to_datetime('07:00:00').time())&((subd['steps'].isna())|(subd['steps']==0))]['hr'].dropna()
        night = subd[(subd['start'].dt.time < pd.to_datetime('07:00:00').time())]['hr'].dropna()
         
        if len(night) >= 10:
            return night_inact.min(), night_inact.std()/np.sqrt(len(night_inact)), night.min(), night.std()/np.sqrt(len(night))
        else:
            return 0,0,0,0 
    else:
        return 0,0,0,0 

In [ ]:
seas = pd.read_csv('mean_rhr_pdps.csv',index_col=0)

In [ ]:
sc = df.source[df.source.notna()].iloc[0]
seas_sc = seas[seas.source == sc].copy()
seas_sc['date'] = pd.to_datetime(seas_sc['date'])
#df = df[['start','rhr','hr','steps','date','day_of_week', 'dt','day_totest', 'week_totest','phase']]

In [ ]:
df_seas = pd.merge(df,seas_sc, how = 'inner', on ='date')

In [ ]:
df_seas['hr_cl'] = df_seas['hr'] - df_seas['mean']
df_seas['rhr_cl'] = df_seas['rhr'] - df_seas['mean']

In [ ]:
df = df[(pd.to_timedelta(df.day_totest).dt.days > -63)&(pd.to_timedelta(df.day_totest).dt.days <= 120)]
wi_list = sliding_window(df['date'].unique(),7)
night_min = []
 
night_std = []
night_inact_min = []
 
night_inact_std = []
 
for wi in wi_list:
    niam, nias, nm, ns = get_min(df_seas[df_seas['date'].isin(wi)])
     
    night_inact_min.append(niam)   
    night_inact_std.append(nias)
    night_min.append(nm)   
    night_std.append(ns)
     

In [ ]:
plt.figure(figsize=(8,4))
 
plt.scatter(pd.to_timedelta(df.day_totest.unique()).days,night_std, s= 2) 
plt.scatter(pd.to_timedelta(df.day_totest.unique()).days,night_inact_std, s= 2) 
plt.axvline(x=0,color='r')  
#plt.xticks(range(len(night_min)),)
#plt.xticks(range(len(night_min))[::30],labels = [str(pd.to_datetime(w[0]).month)+'. '+str(pd.to_datetime(w[0]).year) for w in wi_list][::30],rotation=60);
#plt.ylim(0.05);
#plt.xlim(330)
plt.legend(['all', 'inactive'])
plt.ylabel('HR std ')
plt.xlabel('day to test');

In [ ]:
plt.figure(figsize=(8,4))
 
plt.scatter(pd.to_timedelta(df.day_totest.unique()).days,night_min, s= 2) 
plt.scatter(pd.to_timedelta(df.day_totest.unique()).days,night_inact_min, s= 2) 
plt.axvline(x=0 ,color='r')  
#plt.ylim(40);
#plt.xlim(330)
plt.legend(['all', 'inactive'])
plt.ylabel('HR [bpm]')
plt.xlabel('date');

In [ ]:
df = df.sort_values('date')
df.start = pd.to_datetime(df.start)
d = df.copy()
# take out days where there is no step entry at all 
d_idx = df.groupby('date').sum()[df.groupby('date').sum()['steps'] == 0].index
df['clean'] = df['date'].apply(lambda x: 1 if x in d_idx else 0)
df = df[df['clean'] == 0]
df = df.drop('clean',axis=1)
#
d['steps'] = d['steps'].fillna(0)
d['steps'] = d['steps'] + 1

df_aggv = pd.merge(d[['start','hr','rhr', '% of MHR', 'sleep']].set_index('start').resample('60min').mean().reset_index(),d[['start','steps']].set_index('start').resample('60min').sum().reset_index(),how='outer',on='start')
d_agg = pd.merge(df_aggv, d[['start','date','day_of_week','weekend','dt','day_totest','week_totest', 'phase', 'source','id']].set_index('start').resample('60min').first().reset_index(),how='outer',on='start')

In [ ]:
d_agg['steps'] -= 4

In [ ]:
dph = d_agg[(pd.to_timedelta(d_agg.day_totest).dt.days > -63) &(pd.to_timedelta(d_agg.day_totest).dt.days < 180)]#[(d_agg['phase'] == 1)]
xhd_hr = np.zeros((24,len(dph.day_totest.unique())))
for d in range(len(dph.day_totest.unique())):
    if len(dph[(d_agg['day_totest'] == dph.day_totest.unique()[d] )]) == 24:
        xhd_hr[:, d] = dph[(d_agg['day_totest'] == dph.day_totest.unique()[d] )]['hr'].values

In [ ]:
len(np.where(dph.date.unique() == pd.to_datetime('2022-03-27'))[0]) > 0


In [ ]:
len(np.where(dph.date.unique() == pd.to_datetime('2022-10-30'))[0]) > 0

In [ ]:
plt.figure(figsize=(30,7))
g=sns.heatmap((xhd_hr ) ,   cmap='Blues_r',xticklabels=pd.to_timedelta(dph.day_totest.unique() ).days,
                
                cbar_kws={'label': 'Heart Rate [bpm]'})
g.set_facecolor('xkcd:black')
#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-10-30'))[0][0],c='r')
#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-03-27'))[0][0],c='r')
for xv in np.where(dph.groupby('date').mean().day_of_week == 0)[0]:
    plt.axvline(x=xv, c = 'black')

#plt.legend(['Time Change', r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))
plt.legend([  r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))


plt.tight_layout()
plt.xlabel('Days to test')
plt.ylabel('Hour of day');

In [ ]:
dph = d_agg[(pd.to_timedelta(d_agg.day_totest).dt.days > -63) &(pd.to_timedelta(d_agg.day_totest).dt.days < 180)]#[(d_agg['phase'] == 1)]
xhd = np.zeros((24,len(dph.day_totest.unique())))
for d in range(len(dph.day_totest.unique())):
    if len(dph[(d_agg['day_totest'] == dph.day_totest.unique()[d] )]) == 24:
        xhd[:, d] = dph[(d_agg['day_totest'] == dph.day_totest.unique()[d] )]['steps'].values
        

In [ ]:
len(np.where(dph.date.unique() == pd.to_datetime('2022-03-27'))[0]) > 0


In [ ]:
len(np.where(dph.date.unique() == pd.to_datetime('2022-10-30'))[0]) > 0

In [ ]:
plt.figure(figsize=(30,7))
sns.heatmap((xhd ) ,   cmap='Greens_r',xticklabels=pd.to_timedelta(dph.day_totest.unique() ).days,
                 
                vmin = 0, vmax = 1000,
                #vmin=-1, vmax=np.nanmax(xhd-norm)/norm,
                cbar_kws={'label': 'Steps ','extend':'max',})

#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-10-30'))[0][0],c='r')
#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-03-27'))[0][0],c='r')
for xv in np.where(dph.groupby('date').mean().day_of_week == 0)[0]:
    plt.axvline(x=xv, c = 'grey')
    
#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-03-27'))[0][0],c='r')
    #plt.legend([ r'Week'+'\n'+'Start','DST'], bbox_to_anchor=(1, 1))
plt.legend([  r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))   


#plt.legend(['Time Change', r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))
    
#plt.legend(['DST', r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.xlabel('Days to test')
plt.ylabel('Hour of day');

In [ ]:
mask = (xhd >= 0) & (xhd <= 50)
xhd[mask] = 0
mask = (xhd > 50) & (xhd <= 250)
xhd[mask] = 1
mask = (xhd > 250) & (xhd <= 2000)
xhd[mask] = 2
mask = (xhd > 2000)  
xhd[mask] = 3

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
plt.figure(figsize=(30,7))
cmap_dict = {0: 'grey', 1: 'blue', 2: 'purple', 3: 'red' }
cmap = ListedColormap([cmap_dict[i] for i in range(len(cmap_dict))])
 

ax = sns.heatmap(data=xhd, cmap=cmap, vmin=-0.5, vmax=3.5, yticklabels=True , xticklabels=pd.to_timedelta(dph.day_totest.unique() ).days)
#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-10-30'))[0][0],c='r')
#plt.axvline(x=np.where(dph.date.unique() == pd.to_datetime('2022-03-27'))[0][0],c='r')
for xv in np.where(dph.groupby('date').mean().day_of_week == 0)[0]:
    plt.axvline(x=xv, c = 'black')
    
    
for spine in ax.collections[0].colorbar.ax.spines.values():
    spine.set_visible(True) # show the border of the colorbar
ax.collections[0].colorbar.set_ticklabels(['','0-50 Steps', '','51-250', '','251-2000','','>2000'])
plt.tight_layout()
#plt.legend(['DST', r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))
plt.legend([  r'Week'+'\n'+'Start'], bbox_to_anchor=(1, 1))
plt.xlabel('Days to test')
plt.ylabel('Hour of day');
